# Import Resource Metadata

This notebook provides an example for uploading table descriptions to Secoda. To get started, you will need to [obtain an API key](https://app.secoda.co/settings/api)

Input your API key and Secoda API endpoint URL below. The URL for the cloud instance is `https://api.secoda.co`. If you are self-hosting Secoda or on the EU instance, you will have to update variable.

In [ ]:

import requests

API_KEY = ""
SECODA_API_URL = "https://api.secoda.co"

session = requests.Session()
session.headers.update(dict(
    Authorization=f"Bearer {API_KEY}"
))


def build_url(url: str):
    return f"{SECODA_API_URL}{url}"

## Parse your data

If your data is in CSV format, you could parse it this way and standardize the column names in there to match up with Secoda

In [ ]:
import pandas

dataframe = pandas.read_csv("data/table_descriptions.csv")
dataframe["database"] = dataframe["project"]
dataframe["schema"] = dataframe["dataset_name"]
dataframe["title"] = dataframe["table_name"]
dataframe["description"] = dataframe["table_description"]
dataframe

## Upload descriptions to Secoda

In [ ]:
def find_secoda_table(title, schema, database, *args, **kwargs):
    res = session.get(
        build_url(f"/table/tables/?title={title}&schema={schema}&database={database}")
    )
    results = res.json().get("results")
    if len(results) >= 1:
        return results[0]
    return None

def set_table_description(unique_id, description):
    requests.put(
        build_url(f"/table/tables/{unique_id}"),
        json=dict(
            description=description
        )
    )

for index, entry in enumerate(dataframe.to_dict("records")):
    print(f"Processing {index}/{dataframe.shape[0]}")
    table = find_secoda_table(
        **entry
    )
    if table is not None:
        set_table_description(table.get("id"), entry.get("description", ""))